In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(0)

In [ ]:
def get_volt():
    """Measure voltage.
        - State space model:
            * x_{k+1} = A x_{k} + w_{k}
            * z_{k} = H x_{k} + v_{k}
        - System Model:
            (1) Initial condition:
                x_{0} = 14.4 [V]
                w_{k} = 0
                v_{k} = N(0, 2^{2})
            (2) System condition:
                A (state transition model)              = 1
                H (observation model)                   = 1
                Q (covariance of the process noise)     = 0
                R (covariance of the observation noise) = 4
            (3) State space model:
                x_{k+1} = x_{k}
                z_{k} = x_{k} + v_{k}
    """
    x = 14.4                             # x: state variable.
    v_mean = 0
    v_std = 2
    v = np.random.normal(v_mean, v_std)  # v: measurement noise.
    z = x + v                            # z: observable.
    return z

In [ ]:
def kalman_filter(z_meas, x_esti, P):
    """Estimate voltage using a kalman filter."""
    # (1) Prediction.
    x_pred = A * x_esti
    P_pred = A * P * A + Q

    # (2) Kalman Gain.
    K = P_pred * H / (H * P_pred * H + R)

    # (3) Eastimation.
    x_esti = x_pred + K * (z_meas - H * x_pred)

    # (4) Error Covariance.
    P = P_pred - K * H * P_pred

    return x_esti, P, K

In [ ]:
# Input parameters.
time_start = 0
time_end = 10
time_width = 0.2

In [ ]:
# Initialization for system model.
A = 1
H = 1
Q = 0
R = 4
# Initialization for estimation.
x_0 = 12  # 14 for book.
P_0 = 6
K_0 = 1

In [ ]:
time = np.arange(time_start, time_end, time_width)
n_samples = len(time)

In [ ]:
z_meas_save = np.zeros(n_samples)
x_esti_save = np.zeros(n_samples)
P_save = np.zeros(n_samples)
K_save = np.zeros(n_samples)

In [ ]:
x_esti, P, K = 0, 0, 1
for i in range(n_samples):
    z_meas = get_volt()
    if i == 0:
        x_esti, P, K = x_0, P_0, K_0
    else:
        x_esti, P, K = kalman_filter(z_meas, x_esti, P)

    z_meas_save[i] = z_meas
    x_esti_save[i] = x_esti
    P_save[i] = P
    K_save[i] = K

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(25,10))

plt.subplot(1, 3, 1)
plt.plot(time, z_meas_save, 'r*--', label='Measurements', markersize=15)
plt.plot(time, x_esti_save, 'bo-', label='Kalman Filter', markersize=15)
plt.legend(loc='upper left', fontsize=20)
plt.title('Measurements v.s. Estimation (Kalman Filter)', fontsize=20)
plt.xlabel('Time [sec]', fontsize=25)
plt.ylabel('Voltage [V]', fontsize=25)

plt.subplot(1, 3, 2)
plt.plot(time, P_save, 'go-', markersize=15)
plt.title('Error Covariance (Kalman Filter)', fontsize=20)
plt.xlabel('Time [sec]', fontsize=25)
plt.ylabel('Error Covariance (P)', fontsize=25)

plt.subplot(1, 3, 3)
plt.plot(time, K_save, 'ko-', markersize=15)
plt.title('Kalman Gain (Kalman Filter)', fontsize=20)
plt.xlabel('Time [sec]', fontsize=25)
plt.ylabel('Kalman Gain (K)', fontsize=25)
plt.savefig('png/simple_kalman_filter2.png')